[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Denis-R-V/TSR/blob/main/notebooks/4.1.classifier_make_background.ipynb)

# Система распознавания дорожных знаков на датасете RTSD

## Подготовка ложноположительных детекций для обучения классификатора (класс background)

In [ ]:
# если работаем в колабе - монтируем диск, ставим fiftyone и kaggle
try:
    from google.colab import drive
    drive.mount('/content/drive')

    !pip install fiftyone -q
    !pip install kaggle -q
    
    colab=True
except:
    colab=False

In [ ]:
import json
import os

import torch
import torchvision
from torchvision.transforms import functional as func
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

from PIL import Image
import fiftyone as fo

In [ ]:
# если работаем в колабе - скачиваем и распаковываем датасет
if colab==True:
    !mkdir ~/.kaggle
    !cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json
    
    !kaggle datasets download watchman/rtsd-dataset
    
    !mkdir -p data/raw/RTSD
    !unzip -q rtsd-dataset.zip -d ./data/raw/RTSD/
    !rm rtsd-dataset.zip
    !cp -r data/raw/RTSD/rtsd-frames/rtsd-frames/ data/raw/RTSD
    !rm -r data/raw/RTSD/rtsd-frames/rtsd-frames/

### Пути и параметры

In [2]:
dataset_path = 'data/raw/RTSD' if colab else os.path.join('..', 'data', 'raw', 'RTSD')
models_path = '../models' if colab == False else '../content/drive/MyDrive/TSR/models'
metrics_path = 'data/metrics' if colab else os.path.join('..', 'data', 'metrics')

device_id = 0
if torch.cuda.is_available() == True:
    device = f'cuda:{device_id}'
#elif torch.backends.mps.is_available() == True:
#    device = 'mps'
else:
    device = 'cpu'

#model_name = 'mobilenet_v3_320_b8'          # еще не обучена
#model_name = 'mobilenet_v3_b8'
#model_name = 'resnet50_b8'
#model_name = 'resnet50_v2_b8'
#model_name = 'resnet50_augmented_b8'
#model_name = 'resnet50_v2_augmented_b8'
model_name = 'resnet50_augmented_b16'
#model_name = 'resnet50_v2_augmented_b16'

epoch = 5      # эпоха для загрузки весов

### Загрузка детектора

In [3]:
def load_model_detection(model_name, num_classes, epoch):
    # load model
    if (model_name == 'resnet50_b8') or (model_name == 'resnet50_augmented_b8') or (model_name == 'resnet50_augmented_b16'):
        model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights=None)
        print("Загружена модель resnet50")
    elif (model_name == 'resnet50_v2_b8') or (model_name == 'resnet50_v2_augmented_b8') or (model_name == 'resnet50_v2_augmented_b16'):
        model =torchvision.models.detection.fasterrcnn_resnet50_fpn_v2(weights=None)
        print("Загружена модель resnet50_v2")
    elif model_name == 'mobilenet_v3_b8':
        model =torchvision.models.detection.fasterrcnn_mobilenet_v3_large_fpn(weights=None)
        print("Загружена модель mobilenet_v3")
    elif model_name == 'mobilenet_v3_320_b8':    
        model =torchvision.models.detection.fasterrcnn_mobilenet_v3_large_320_fpn(weights=None)
        print("Загружена модель mobilenet_v3_320")
    else:
        print("Неверно указано название модели")
    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    
    # Загрузка весов модели
    checkpoint = torch.load(os.path.join(models_path, f'detector_{model_name}_chkpt_{epoch}.pth'), map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    print(f"Для модели {model_name} загружены веса эпохи {epoch}")
    model.eval()
    return model

model = load_model_detection(model_name, num_classes=2, epoch=epoch).to(device)

Загружена модель resnet50
Для модели resnet50_augmented_b16 загружены веса эпохи 5


### Загрузка обучающего датасета (reduced)

In [4]:
#Load train dataset COCO

# The directiry containing the source images
data_path = dataset_path      # можно не указывать, если в JSON путь совпадает

# The path yo the COCO labels JSON file
train_anno_reduced_file = "train_anno_reduced_bin_class.json"
val_anno_path = os.path.join(dataset_path, train_anno_reduced_file)

# Import the dataset
train_reduced_dataset = fo.Dataset.from_dir(
    dataset_type=fo.types.COCODetectionDataset,
    data_path=data_path,
    labels_path=val_anno_path
)

 100% |███████████████| 1889/1889 [2.4s elapsed, 0s remaining, 795.7 samples/s]      


In [ ]:
#predictions_view = train_reduced_dataset.take(50, seed=42)

In [5]:
# Add and evaluating Faster-RCNN detections
# Faster-RCNN detections can be calculated and added to every sample of the dataset in a new field

#classes = predictions_view.default_classes
classes = train_reduced_dataset.default_classes
# Add predictions
#for sample in predictions_view:
model.eval()
for i, sample in enumerate(train_reduced_dataset):
    image = Image.open(sample.filepath)
    image = func.to_tensor(image).to(device)
    c,h,w = image.shape
    preds = model([image])[0]
    labels = preds['labels'].cpu().detach().numpy()
    scores = preds['scores'].cpu().detach().numpy()
    boxes = preds['boxes'].cpu().detach().numpy()
    detections = []
    for label, score, box in zip (labels, scores, boxes):
        # Compute relative bounding box coordinates
        x1, y1, x2, y2 = box
        rel_box = [x1/w, y1/h, (x2-x1)/w, (y2-y1)/h]
        detections.append(fo.Detection(label=classes[label],
                                        bounding_box=rel_box,
                                        confidence=score
                                        )
                            )
    sample['faster_rcnn'] = fo.Detections(detections=detections)
    sample.save()
    if i % 100 == 0:
            print(f"\tОбработано {i} из {train_reduced_dataset.__len__()} сэмплов.")

	Обработано 0 из 1889 сэмплов.
	Обработано 100 из 1889 сэмплов.
	Обработано 200 из 1889 сэмплов.
	Обработано 300 из 1889 сэмплов.
	Обработано 400 из 1889 сэмплов.
	Обработано 500 из 1889 сэмплов.
	Обработано 600 из 1889 сэмплов.
	Обработано 700 из 1889 сэмплов.
	Обработано 800 из 1889 сэмплов.
	Обработано 900 из 1889 сэмплов.
	Обработано 1000 из 1889 сэмплов.
	Обработано 1100 из 1889 сэмплов.
	Обработано 1200 из 1889 сэмплов.
	Обработано 1300 из 1889 сэмплов.
	Обработано 1400 из 1889 сэмплов.
	Обработано 1500 из 1889 сэмплов.
	Обработано 1600 из 1889 сэмплов.
	Обработано 1700 из 1889 сэмплов.
	Обработано 1800 из 1889 сэмплов.


In [6]:
# Evaluate the predictions in the `faster_rcnn` field of our `high_conf_view`
# with respect to the objects in the `ground_truth` field
results = train_reduced_dataset.evaluate_detections(
    "faster_rcnn",
    gt_field="detections",
    eval_key="eval",
    compute_mAP=True,
)

Evaluating detections...
 100% |███████████████| 1889/1889 [8.2s elapsed, 0s remaining, 270.0 samples/s]       
Performing IoU sweep...
 100% |███████████████| 1889/1889 [8.3s elapsed, 0s remaining, 248.7 samples/s]      


In [10]:
# Для faster_rcnn оставляем только FP детекции
fp_detections = train_reduced_dataset.filter_labels("faster_rcnn", fo.ViewField("eval") == "fp")

In [ ]:
session = fo.launch_app(fp_detections)

In [13]:
# Сохраняем файл с аннотацией FP Детекций
fp_detections.export(
    export_dir=os.path.join(dataset_path),
    dataset_type=fo.types.COCODetectionDataset,
    labels_path='train_anno_reduced_fp_detections.json',
    label_field='faster_rcnn',
    )

Directory 'data' already exists; export will be merged with existing files
 100% |███████████████| 1576/1576 [5.1s elapsed, 0s remaining, 348.1 samples/s]      


In [14]:
# Поменяем ссылки на существующие изображения в файле аннотации
# Каталог с сохраненными изображениями (dataset_path/data) можено удалить

with open(os.path.join(dataset_path, 'train_anno_reduced_fp_detections.json'), 'r') as read_file:
    train_anno_reduced_fp_detections = json.load(read_file)
read_file.close()
train_anno_reduced_fp_detections['images']

for i in range(len(train_anno_reduced_fp_detections['images'])):
    train_anno_reduced_fp_detections['images'][i]['file_name'] = 'rtsd-frames/' + train_anno_reduced_fp_detections['images'][i]['file_name']
train_anno_reduced_fp_detections['images']

with open(os.path.join(dataset_path, 'train_anno_reduced_fp_detections.json'), 'w') as f:
    json.dump(train_anno_reduced_fp_detections, f)#ensure_ascii=False, indent=4)
f.close()

In [16]:
# Смотрим итоговую аннотацию
with open(os.path.join(dataset_path, 'train_anno_reduced_fp_detections.json'), 'r') as read_file:
    train_anno_reduced_fp_detections = json.load(read_file)
read_file.close()
train_anno_reduced_fp_detections['images']

[{'id': 1,
  'file_name': 'rtsd-frames/autosave01_02_2012_09_16_49.jpg',
  'height': 720,
  'width': 1280,
  'license': None,
  'coco_url': None},
 {'id': 2,
  'file_name': 'rtsd-frames/autosave01_02_2012_09_16_50.jpg',
  'height': 720,
  'width': 1280,
  'license': None,
  'coco_url': None},
 {'id': 3,
  'file_name': 'rtsd-frames/autosave01_02_2012_09_16_51.jpg',
  'height': 720,
  'width': 1280,
  'license': None,
  'coco_url': None},
 {'id': 4,
  'file_name': 'rtsd-frames/autosave01_02_2012_09_16_56.jpg',
  'height': 720,
  'width': 1280,
  'license': None,
  'coco_url': None},
 {'id': 5,
  'file_name': 'rtsd-frames/autosave01_02_2012_09_17_01.jpg',
  'height': 720,
  'width': 1280,
  'license': None,
  'coco_url': None},
 {'id': 6,
  'file_name': 'rtsd-frames/autosave01_02_2012_09_17_04.jpg',
  'height': 720,
  'width': 1280,
  'license': None,
  'coco_url': None},
 {'id': 7,
  'file_name': 'rtsd-frames/autosave01_02_2012_09_17_05.jpg',
  'height': 720,
  'width': 1280,
  'license'

### Отбор реальных FP детекций

Загрузка датасета с FP детекциями

In [17]:
# загрузка датасета

# The directiry containing the source images
data_path = dataset_path     # можно не указывать, если в JSON путь совпадает

# The path yo the COCO labels JSON file
labels_file = 'train_anno_reduced_fp_detections.json'
labels_path = os.path.join(dataset_path, labels_file)


# Import the dataset
fp_detections_dataset = fo.Dataset.from_dir(
    dataset_type=fo.types.COCODetectionDataset,
    data_path=data_path,
    labels_path=labels_path
)

 100% |███████████████| 1576/1576 [4.6s elapsed, 0s remaining, 363.5 samples/s]      


In [ ]:
# Визуализация набора данных
# Для FP детекций (пойдут в фон классификатора) необходимо добавить label 'background'

session = fo.launch_app(fp_detections_dataset)

In [19]:
fp_detections_dataset.save

<bound method Dataset.save of Name:        2023.01.22.20.20.26
Media type:  image
Num samples: 1576
Persistent:  False
Tags:        []
Sample fields:
    id:         fiftyone.core.fields.ObjectIdField
    filepath:   fiftyone.core.fields.StringField
    tags:       fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:   fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    detections: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)>

In [20]:
for sample in fp_detections_dataset:
    detections = []
    for i in range(len(sample.detections.detections)):
        if 'background' in sample.detections.detections[i]['tags']:
            label = sample.detections.detections[i]['label']
            box = sample.detections.detections[i]['bounding_box']
            score = sample.detections.detections[i]['confidence']
            detections.append(fo.Detection(label=label,
                                        bounding_box=box,
                                        confidence=score
                                        )
                            )
        sample['background'] = fo.Detections(detections=detections)
        sample.save()

In [23]:
# Сохраняем файл с аннотацией FP Детекций
dataset_background = fp_detections_dataset.filter_labels('background', fo.ViewField("label") == "sign")
dataset_background.export(
    export_dir=os.path.join(dataset_path),
    dataset_type=fo.types.COCODetectionDataset,
    labels_path='train_anno_reducer_background.json',
    label_field='background',
)

Directory 'data' already exists; export will be merged with existing files
 100% |█████████████████| 590/590 [1.8s elapsed, 0s remaining, 351.8 samples/s]         


In [29]:
# Поменяем ссылки на существующие изображения в файле аннотации
# Каталог с сохраненными изображениями (dataset_path/data) можено удалить

with open(os.path.join(dataset_path, 'train_anno_reducer_background.json'), 'r') as read_file:
    train_anno_reducer_background = json.load(read_file)
read_file.close()
train_anno_reducer_background['images']

for i in range(len(train_anno_reducer_background['images'])):
    train_anno_reducer_background['images'][i]['file_name'] = 'rtsd-frames/' + train_anno_reducer_background['images'][i]['file_name']
train_anno_reducer_background['images']

with open(os.path.join(dataset_path, 'train_anno_reducer_background.json'), 'w') as f:
    json.dump(train_anno_reducer_background, f)#ensure_ascii=False, indent=4)
f.close()

In [32]:
# загрузка датасета

# The directiry containing the source images
data_path = dataset_path      # можно не указывать, если в JSON путь совпадает

# The path yo the COCO labels JSON file
labels_file = "train_anno_reducer_background.json"
labels_path = os.path.join(dataset_path, labels_file)


# Import the dataset
dataset_background = fo.Dataset.from_dir(
    dataset_type=fo.types.COCODetectionDataset,
    data_path=data_path,
    labels_path=labels_path
)

 100% |█████████████████| 590/590 [822.8ms elapsed, 0s remaining, 717.0 samples/s]      


In [ ]:
session = fo.launch_app(dataset_background)

### Загрузка валидационного датасета

In [6]:
#Load val dataset COCO

# The directiry containing the source images
data_path = dataset_path      # можно не указывать, если в JSON путь совпадает

# The path yo the COCO labels JSON file
val_anno_file = "val_anno_bin_class.json"
val_anno_path = os.path.join(dataset_path, val_anno_file)

# Import the dataset
val_dataset = fo.Dataset.from_dir(
    dataset_type=fo.types.COCODetectionDataset,
    data_path=data_path,
    labels_path=val_anno_path
)

 100% |███████████████| 5000/5000 [7.9s elapsed, 0s remaining, 704.7 samples/s]      


In [ ]:
#predictions_view = val_dataset.take(50, seed=42)

In [7]:
# Add and evaluating Faster-RCNN detections
# Faster-RCNN detections can be calculated and added to every sample of the dataset in a new field

#classes = predictions_view.default_classes
classes = val_dataset.default_classes
# Add predictions
#for sample in predictions_view:
model.eval()
for i, sample in enumerate(val_dataset):
    image = Image.open(sample.filepath)
    image = func.to_tensor(image).to(device)
    c,h,w = image.shape
    preds = model([image])[0]
    labels = preds['labels'].cpu().detach().numpy()
    scores = preds['scores'].cpu().detach().numpy()
    boxes = preds['boxes'].cpu().detach().numpy()
    detections = []
    for label, score, box in zip (labels, scores, boxes):
        # Compute relative bounding box coordinates
        x1, y1, x2, y2 = box
        rel_box = [x1/w, y1/h, (x2-x1)/w, (y2-y1)/h]
        detections.append(fo.Detection(label=classes[label],
                                        bounding_box=rel_box,
                                        confidence=score
                                        )
                            )
    sample['faster_rcnn'] = fo.Detections(detections=detections)
    sample.save()
    if i % 100 == 0:
            print(f"\tОбработано {i} из {val_dataset.__len__()} сэмплов.")

	Обработано 0 из 5000 сэмплов.
	Обработано 100 из 5000 сэмплов.
	Обработано 200 из 5000 сэмплов.
	Обработано 300 из 5000 сэмплов.
	Обработано 400 из 5000 сэмплов.
	Обработано 500 из 5000 сэмплов.
	Обработано 600 из 5000 сэмплов.
	Обработано 700 из 5000 сэмплов.
	Обработано 800 из 5000 сэмплов.
	Обработано 900 из 5000 сэмплов.
	Обработано 1000 из 5000 сэмплов.
	Обработано 1100 из 5000 сэмплов.
	Обработано 1200 из 5000 сэмплов.
	Обработано 1300 из 5000 сэмплов.
	Обработано 1400 из 5000 сэмплов.
	Обработано 1500 из 5000 сэмплов.
	Обработано 1600 из 5000 сэмплов.
	Обработано 1700 из 5000 сэмплов.
	Обработано 1800 из 5000 сэмплов.
	Обработано 1900 из 5000 сэмплов.
	Обработано 2000 из 5000 сэмплов.
	Обработано 2100 из 5000 сэмплов.
	Обработано 2200 из 5000 сэмплов.
	Обработано 2300 из 5000 сэмплов.
	Обработано 2400 из 5000 сэмплов.
	Обработано 2500 из 5000 сэмплов.
	Обработано 2600 из 5000 сэмплов.
	Обработано 2700 из 5000 сэмплов.
	Обработано 2800 из 5000 сэмплов.
	Обработано 2900 из 5000 с

In [10]:
# Evaluate the predictions in the `faster_rcnn` field of our `high_conf_view`
# with respect to the objects in the `ground_truth` field
results = val_dataset.evaluate_detections(
    "faster_rcnn",
    gt_field="detections",
    eval_key="eval",
    compute_mAP=True,
)

Evaluating detections...
 100% |███████████████| 5000/5000 [20.8s elapsed, 0s remaining, 289.2 samples/s]      
Performing IoU sweep...
 100% |███████████████| 5000/5000 [21.6s elapsed, 0s remaining, 262.6 samples/s]      


In [32]:
# Для faster_rcnn оставляем только FP детекции
fp_detections = val_dataset.filter_labels("faster_rcnn", fo.ViewField("eval") == "fp")

In [ ]:
session = fo.launch_app(fp_detections)

In [50]:
# Сохраняем файл с аннотацией FP Детекций
fp_detections.export(
    export_dir=os.path.join(dataset_path),
    dataset_type=fo.types.COCODetectionDataset,
    labels_path='val_anno_fp_detections.json',
    label_field='faster_rcnn',
    )
    

Directory 'data' already exists; export will be merged with existing files
 100% |███████████████| 2803/2803 [8.4s elapsed, 0s remaining, 342.9 samples/s]       


In [51]:
# Поменяем ссылки на существующие изображения в файле аннотации
# Каталог с сохраненными изображениями (dataset_path/data) можено удалить

with open(os.path.join(dataset_path, 'val_anno_fp_detections.json'), 'r') as read_file:
    val_anno_fp_detections = json.load(read_file)
read_file.close()
val_anno_fp_detections['images']

for i in range(len(val_anno_fp_detections['images'])):
    val_anno_fp_detections['images'][i]['file_name'] = 'rtsd-frames/' + val_anno_fp_detections['images'][i]['file_name']
val_anno_fp_detections['images']

with open(os.path.join(dataset_path, 'val_anno_fp_detections.json'), 'w') as f:
    json.dump(val_anno_fp_detections, f)#ensure_ascii=False, indent=4)
f.close()

In [53]:
# Смотрим итоговую аннотацию
with open(os.path.join(dataset_path, 'val_anno_fp_detections.json'), 'r') as read_file:
    val_anno_fp_detections = json.load(read_file)
read_file.close()
val_anno_fp_detections['images']

[{'id': 1,
  'file_name': 'rtsd-frames/autosave01_02_2012_09_14_07.jpg',
  'height': 720,
  'width': 1280,
  'license': None,
  'coco_url': None},
 {'id': 2,
  'file_name': 'rtsd-frames/autosave01_02_2012_09_15_18.jpg',
  'height': 720,
  'width': 1280,
  'license': None,
  'coco_url': None},
 {'id': 3,
  'file_name': 'rtsd-frames/autosave01_02_2012_09_16_55.jpg',
  'height': 720,
  'width': 1280,
  'license': None,
  'coco_url': None},
 {'id': 4,
  'file_name': 'rtsd-frames/autosave01_02_2012_09_17_00.jpg',
  'height': 720,
  'width': 1280,
  'license': None,
  'coco_url': None},
 {'id': 5,
  'file_name': 'rtsd-frames/autosave01_02_2012_09_18_30.jpg',
  'height': 720,
  'width': 1280,
  'license': None,
  'coco_url': None},
 {'id': 6,
  'file_name': 'rtsd-frames/autosave01_02_2012_09_18_48.jpg',
  'height': 720,
  'width': 1280,
  'license': None,
  'coco_url': None},
 {'id': 7,
  'file_name': 'rtsd-frames/autosave01_02_2012_09_18_49.jpg',
  'height': 720,
  'width': 1280,
  'license'

### Отбор реальных FP детекций

Загрузка датасета с FP детекциями

In [11]:
# загрузка датасета

# The directiry containing the source images
data_path = dataset_path     # можно не указывать, если в JSON путь совпадает

# The path yo the COCO labels JSON file
labels_file = 'val_anno_fp_detections.json'
labels_path = os.path.join(dataset_path, labels_file)


# Import the dataset
fp_detections_dataset = fo.Dataset.from_dir(
    dataset_type=fo.types.COCODetectionDataset,
    data_path=data_path,
    labels_path=labels_path
)

 100% |███████████████| 2803/2803 [5.6s elapsed, 0s remaining, 530.8 samples/s]      


In [ ]:
# Визуализация набора данных
# Для FP детекций (пойдут в фон классификатора) необходимо добавить label 'background'

session = fo.launch_app(fp_detections_dataset)

In [14]:
fp_detections_dataset.save

<bound method Dataset.save of Name:        2023.01.22.17.35.47
Media type:  image
Num samples: 2803
Persistent:  False
Tags:        []
Sample fields:
    id:         fiftyone.core.fields.ObjectIdField
    filepath:   fiftyone.core.fields.StringField
    tags:       fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:   fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    detections: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)>

In [15]:
for sample in fp_detections_dataset:
    detections = []
    for i in range(len(sample.detections.detections)):
        if 'background' in sample.detections.detections[i]['tags']:
            label = sample.detections.detections[i]['label']
            box = sample.detections.detections[i]['bounding_box']
            score = sample.detections.detections[i]['confidence']
            detections.append(fo.Detection(label=label,
                                        bounding_box=box,
                                        confidence=score
                                        )
                            )
        sample['background'] = fo.Detections(detections=detections)
        sample.save()

In [106]:
fp_detections_dataset.export(
    export_dir=os.path.join(dataset_path),
    dataset_type=fo.types.COCODetectionDataset,
    labels_path='val_anno_background.json',
    label_field='background',
    )

Directory 'data' already exists; export will be merged with existing files
Found multiple fields ['detections', 'background'] with compatible type (<class 'fiftyone.core.labels.Detections'>, <class 'fiftyone.core.labels.Polylines'>, <class 'fiftyone.core.labels.Keypoints'>); exporting 'detections'


In [16]:
# Сохраняем файл с аннотацией FP Детекций
dataset_background = fp_detections_dataset.filter_labels('background', fo.ViewField("label") == "sign")
dataset_background.export(
    export_dir=os.path.join(dataset_path),
    dataset_type=fo.types.COCODetectionDataset,
    labels_path='val_anno_background.json',
    label_field='background',
)

Directory 'data' already exists; export will be merged with existing files
 100% |███████████████| 1470/1470 [4.2s elapsed, 0s remaining, 436.7 samples/s]      


In [17]:
dataset_background

Dataset:     2023.01.22.17.35.47
Media type:  image
Num samples: 1470
Sample fields:
    id:         fiftyone.core.fields.ObjectIdField
    filepath:   fiftyone.core.fields.StringField
    tags:       fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:   fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    detections: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    background: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
View stages:
    1. FilterLabels(field='background', filter={'$eq': ['$$this.label', 'sign']}, only_matches=True, trajectories=False)

In [18]:
with open(os.path.join(dataset_path, 'val_anno_background.json'), 'r') as read_file:
    label_map = json.load(read_file)
read_file.close()
label_map['annotations'][-1]

{'id': 2082,
 'image_id': 1470,
 'category_id': 0,
 'bbox': [941.7908935546875,
  429.67187499999994,
  20.73358154296875,
  19.98858642578125],
 'score': 0.06526462733745575,
 'area': 414.4349865876138,
 'iscrowd': 0}

In [35]:
# Поменяем ссылки на существующие изображения в файле аннотации
# Каталог с сохраненными изображениями (dataset_path/data) можено удалить

with open(os.path.join(dataset_path, 'val_anno_background.json'), 'r') as read_file:
    val_anno_background = json.load(read_file)
read_file.close()
val_anno_background['images']

for i in range(len(val_anno_background['images'])):
    val_anno_background['images'][i]['file_name'] = 'rtsd-frames/' + val_anno_background['images'][i]['file_name']
val_anno_background['images']

with open(os.path.join(dataset_path, 'val_anno_background.json'), 'w') as f:
    json.dump(val_anno_background, f)#ensure_ascii=False, indent=4)
f.close()
val_anno_background['images']

In [37]:
# загрузка датасета

# The directiry containing the source images
data_path = dataset_path      # можно не указывать, если в JSON путь совпадает

# The path yo the COCO labels JSON file
labels_file = "val_anno_background.json"
labels_path = os.path.join(dataset_path, labels_file)


# Import the dataset
dataset_background = fo.Dataset.from_dir(
    dataset_type=fo.types.COCODetectionDataset,
    data_path=data_path,
    labels_path=labels_path
)

 100% |███████████████| 1470/1470 [2.0s elapsed, 0s remaining, 714.8 samples/s]      


In [ ]:
# Визуализация набора данных
session = fo.launch_app(dataset_background)

In [22]:
dataset_background

Name:        2023.01.22.19.58.17
Media type:  image
Num samples: 1470
Persistent:  False
Tags:        []
Sample fields:
    id:         fiftyone.core.fields.ObjectIdField
    filepath:   fiftyone.core.fields.StringField
    tags:       fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:   fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    detections: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)